In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('preprocessed.csv', index_col = 0)

In [ ]:
df.head()


In [ ]:
ax_data = df['ax(g)']

In [ ]:
ax_data.head()

In [ ]:
train_size = int(len(ax_data) * 0.8)

In [ ]:
test_size = len(ax_data) - train_size

In [ ]:
train, test = ax_data[0:train_size], ax_data[train_size:len(df)]

In [ ]:
train.head()

In [ ]:
print(len(train), len(test))

In [ ]:
def create_dataset(dataset,look_back=1):
    Xs, ys = [], []
    dataset = np.array(dataset)
    for i in range(len(dataset) - look_back - 1):
        v= dataset[i:(i + look_back)]
        Xs.append(v)
        ys.append(dataset[(i + look_back)])
    return np.array(Xs), np.array(ys)

In [ ]:
look_back = 10

In [ ]:
train.shape

In [ ]:
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

print(X_train.shape, y_train.shape)

In [ ]:
# X_train[1:5]
X_test[1:5]

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],1,X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0],1,X_test.shape[1]))

In [ ]:
X_train[-5:]

# **Modelling Using LSTM**

In [ ]:
model = keras.Sequential()
model.add(keras.layers.LSTM(units = 128, input_shape = (1,look_back)))


In [ ]:
model.add(keras.layers.Dense(units=1))
model.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(0.001))
model.fit(X_train[1:1000], y_train[1:1000], epochs = 2, batch_size=1, verbose=2)

In [ ]:
train_predict = model.predict(X_train[1:10000])
test_predict = model.predict(X_test[1:10000])


In [ ]:
print('Train Mean Absolute Error:', keras.losses.MAE(y_train[0], train_predict[:,0]))
print('Train Root Mean Squared Error:',np.sqrt(keras.losses.MSE(y_train[0], train_predict[:,0])))
print('Test Mean Absolute Error:', keras.losses.MAE(y_test[0], test_predict[:,0]))
print('Test Root Mean Squared Error:',np.sqrt(keras.losses.MSE(y_test[0], test_predict[:,0])))

In [ ]:
aa=[x for x in range(200)]
plt.figure(figsize=(8,4))
plt.plot(aa, y_test[:200], marker='.', label="actual")
plt.plot(aa, test_predict[:,0][:200], 'r', label="prediction")
# plt.tick_params(left=False, labelleft=True) #remove ticks
plt.tight_layout()
plt.subplots_adjust(left=0.07)
plt.ylabel('ax(g)', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)
plt.show();